In [1]:
system("sudo apt-get update -y && sudo apt-get install -y jags && sudo apt-get install -y r-cran-rjags")
install.packages('runjags')
install.packages('rjags')

In [2]:
# Charger les bibliothèques nécessaires
library(dplyr)
library(ggplot2)
library(tidyverse)
library(runjags)
library(rjags)

# Introduction
L’étude des disparités salariales est un sujet central en économie du travail et en sciences sociales. Malgré les avancées vers l’égalité des chances, des différences de salaires persistent entre divers groupes sociaux, notamment en fonction du genre, de l’ethnicité, du niveau d’éducation et de l’expérience professionnelle. Aux États-Unis, des études ont montré que les écarts salariaux sont significatifs : les femmes gagnent en moyenne moins que les hommes, et certaines minorités ethniques, comme les Afro-Américains et les Latino-Américains, perçoivent des salaires inférieurs à ceux des Américains blancs et asiatiques.

Notre projet vise à identifier les principaux facteurs influençant les salaires horaires en utilisant des modèles bayésiens. Plus précisément, nous cherchons à quantifier l’impact de variables telles que le genre, l’expérience professionnelle, le niveau d’éducation, l’appartenance syndicale et la localisation géographique sur les salaires.

Pour ce faire, nous utiliserons des méthodes bayésiennes de régression mises en œuvre avec JAGS, permettant une estimation robuste et une meilleure quantification des incertitudes. Notre approche inclura l’exploration des données, la spécification des modèles avec des choix de lois a priori justifiées, ainsi que des évaluations rigoureuses des performances des modèles à travers des diagnostics de convergence et des simulations de données fictives (fake data check).

Ce rapport est structuré comme suit : après une présentation des données et de leur exploration, nous décrirons les modèles bayésiens retenus et justifierons les choix de lois a priori. Nous présenterons ensuite les résultats de l’ajustement des modèles, suivis des diagnostics de convergence et de l’évaluation de leur performance. Enfin, nous discuterons des conclusions et des implications de nos résultats dans le contexte des disparités salariales aux États-Unis.

# Modèle génératif et espace des paramètres

On veut construire un modèle prédictif pour le salaire des individus pour lesquels on connait 10 variables explicatives (âge, éducation, experience professionnelle, etc.), 434 individus dans un jeu de données d'entraînement et 100 individus dans le jeu de données de test.

On note $Y_i \in \mathbb{R}$ le salaire de l'individu $i$ et $X_i \in \mathbb{R}^{10}$ les variables explicatives de l'individu $i$.

Régression linéaire:
$Y_i = \beta_0 + \beta_1 X_{1i} + \beta_2 X_{2i} + \cdots + \beta_{10} X_{10i} + \sigma \epsilon_i$
11 paramètres $\beta = (\beta_0, \beta_1, \beta_2, \cdots, \beta_{10})$  $\epsilon_i \sim \mathcal{N}(0, 1)$ et $\sigma \in \mathbb{R^+}$

Certaines variables peuvent avoir peu ou pas d'effet sur le salaire, ie des $\beta_i = 0$.

## Distribution à postériori

A partir de $y=(y_1, y_2, \cdots, y_{434})$ et $ X=(X_1, X_2, \cdots, X_{434})$, on peut calculer $P(\beta | y, X)$ et regarder $P(\beta_j \neq 0 | y, X)$ pour chaque coefficient.

## Performances prédictives

Un manière d'évaluer la performance prédictive $\hat{\beta}_{bayes} = E(\beta | y, X)$
$X_{test}$ matrice 100x10
$y_{test}$ les observations
$y_{test,Bayes} = X_{test} \hat{\beta}_{bayes}$

# Degrès de croyance

Que croit-on sur l'effet de chaque variable explicative sur le salaire?

Nous pouvons formuler des hypothèse à priori sur l'impact des différentes variables :

- Éducation : On s’attend à une relation positive entre le niveau d’éducation et le salaire.

- Expérience professionnelle (workexp) : Plus d’expérience est généralement associée à un salaire plus élevé.

- Sexe (female) : Il est bien documenté qu’un écart salarial entre hommes et femmes existe (gender pay gap).

- Région (south) : Il peut y avoir des différences salariales régionales.

- Appartenance syndicale (unionmember) : Être syndiqué peut influencer le salaire.

- Âge (age) : En général, l’âge est corrélé à l’expérience professionnelle, mais il pourrait aussi capturer d’autres dynamiques comme la discrimination liée à l’âge.

- Ethnicité (ethnicity) : Certaines inégalités salariales peuvent exister selon l’origine ethnique.

- Occupation et secteur (occupation, sector) : Le type d’emploi et le secteur d’activité influencent fortement le salaire.

- Statut matrimonial (married) : Certaines études suggèrent que les personnes mariées ont des salaires plus élevés, en particulier les hommes.


### Estimation d'un paramètre $\beta$ avec des données

Ayant obtenu {$Y=y$} , $P(\beta | y,X) = \frac{P(\beta) P(y,X | \beta)}{P(y,X)}$

Donc $posterior \propto prior \times likelihood$

$prior = P(\beta)$ : Croyance des effets des variables sur le salaire avant d'avoir les données.
$likelihood = P(y | \beta)$ : Probabilité d'observer les données en fonction des paramètres du modèle
$posterior = P(\beta | y)$ : La Distribution à posteriori après avoir observé les données

On peut le voir ainsi :

$P(Model | New Data)  = \frac{P(Model) P(New Data | Model)}{P(New Data)}$



# Nos données

Notre jeu de données comprend 534 individus. Chaque individu a 10 variables : education, south, female, workexp, unionmember, wages, age, ethnicity, occupation, sector, married.

- Variables quantitatives:
    > **education**: Années d'études
    >**workexp**: Années d'expêrience professionnelle
    >**age**: âge

- Variables qualitatives:
    >**south**: Vit dans le sud ou non
    >**female**: Indicateur de genre (0 = Homme, 1 = Femme)
    >**unionmember**: Indicateur de membre du syndicat (0 = Non, 1 = Oui)
    >**married**: Indicateur de statut marital. Marié(e) ou non.

- Variables categorielles:
    >**ethnicity**: Ethnicité (White, Hispanic, Other)
    >**occupation**: Occupation (Management, Sales, Clerical, Professional, Service, Other)
    >**sector**: Secteur d'activité (Manufactoring, Construction, Other)




# Modélisation Bayésienne

* L'analyse traditionnelle des déterminants salariaux repose souvent sur des modèles de régression visant à prédire le salaire horaire individuel comme variable continue. Cependant, cette approche présente une limite majeure : elle ne permet pas d'identifier clairement les facteurs qui influencent l'appartenance à une catégorie significative de salaire (par exemple, un revenu "élevé" versus "bas"). En effet, prédire une valeur exacte de salaire horaire (par exemple, 15,67 $/h) n’est pas toujours utile pour comprendre les dynamiques structurelles des inégalités ou pour élaborer des politiques ciblées.
* Pour pallier cette limite, nous avons adopté une approche alternative : transformer la variable salariale continue en une variable binaire indiquant si un individu perçoit un salaire supérieur au salaire moyen (1) ou non (0). Cette 
* Pour ce faire, on ajoute une colonne pour calculer la moyenne de la variable wage avec 0 pour les salaires inférieurs à la moyenne et 1 pour les salaires supérieurs à la moyenne.

In [167]:
# Charger les données
wage_data <- read.csv("https://raw.githubusercontent.com/derghalmanal/Wage/refs/heads/main/wage_data.csv", header=TRUE, sep=",")

# Calculer la moyenne de la colonne wage
mean_wage <- mean(wage_data$wage, na.rm=TRUE)

# Ajouter la colonne wage_moyenne
wage_data$wage_level <- ifelse(wage_data$wage < mean_wage, 0, 1)

# Transformer la colonne wage_moyenne en facteur
wage_data$wage_level <- as.factor(wage_data$wage_level)

# Vérifier le résultat
head(wage_data)

In [168]:
str(wage_data)

Analysons la contribution de chaque variable aux salaires

Dans toute la suite du rapport l = lower pour signifier que le salaire est inférieur à la moyenne et h = higher pour signifier que le salaire est supérieur à la moyenne.

On cherchera la loi de la vraisemblance $X_{i,s}|\beta$ pour chaque individu $i$ et chaque variable $s \in \{l,h\}$.

In [169]:
plot_credible_intervals <- function(jags_samples) {
  # Extract the MCMC samples and the names of the parameters
  samples <- as.data.frame(jags_samples$mcmc %>% lapply(as_tibble) %>% bind_rows())
  params <- names(samples)

  # Calculate the 50% and 95% credible intervals for each parameter
  intervals <- data.frame(param = character(), lower = numeric(), upper = numeric())
  for (param in params) {
    est_mean <- mean(samples[[param]])
    est_median <- median(samples[[param]])
    ci_50_infCI <- quantile(samples[[param]], probs = 0.25)
    ci_50_supCI <- quantile(samples[[param]], probs = 0.75)
    ci_95_infCI <- quantile(samples[[param]], probs = 0.025)
    ci_95_supCI <- quantile(samples[[param]], probs = 0.975)
    intervals <- rbind(intervals, data.frame(param = param, est_mean = est_mean, est_median = est_median, ci_50_infCI = ci_50_infCI, ci_50_supCI = ci_50_supCI, ci_95_infCI = ci_95_infCI, ci_95_supCI = ci_95_supCI))
  }

  # Create a ggplot object
  p <- ggplot(intervals, aes(y = param)) +
    theme_classic() +
    geom_segment(aes(y = param, yend = param, x = ci_95_infCI, xend = ci_95_supCI),
      color = "red", size = 0.5
    ) +
    geom_segment(aes(y = param, yend = param, x = ci_50_infCI, xend = ci_50_supCI),
      color = "red", size = 1.5
    ) +
    geom_point(aes(x = est_mean), size = 3) +
    labs(title = "Posterior credible intervals") +
    xlab("") +
    ylab("")

  # Print the plot
  print(p)
}

# Histogramme des variables

In [170]:
numeric_vars <- c("education", "workexp")
wage_data[numeric_vars] <- lapply(wage_data[numeric_vars], function(x) as.numeric(as.character(x)))

for (var in numeric_vars){
    print(ggplot(data = wage_data, aes_string(x = var, fill = "wage_level" )) +
      geom_bar( position = "identity", alpha = 0.7) +
      labs(title = paste0("Histogramme de la variable ", var , " en fonction du salaire moyen"),
           x = var, y = "Fréquence") +
      scale_fill_manual(values = c("blue", "red")) +  # Couleurs des labels
      theme_minimal())
}

wage_data$south        <- factor(wage_data$south, labels = c("Non-South", "South"))
wage_data$female       <- factor(wage_data$female, labels = c("Male", "Female"))
wage_data$unionmember  <- factor(wage_data$unionmember, labels = c("Non-Member", "Member"))
wage_data$married      <- factor(wage_data$married, labels = c("Single", "Married"))

# Now binary_vars are factors with clear labels
binary_vars <- c("south", "female", "unionmember", "married")

# Plot
for (var in binary_vars){
  print(
    ggplot(data = wage_data, aes_string(x = var, fill = "wage_level")) +
      geom_bar(position = "dodge", alpha = 0.7) +
      labs(title = paste("Histogramme de la variable", var, "en fonction du niveau de salaire"),
           x = var, y = "Fréquence") +
      scale_fill_manual(values = c("blue", "red"), 
                        labels = c("Salaire < Moyenne", "Salaire ≥ Moyenne")) +
      theme_minimal()
  )
}
categorial_vars <- c("ethnicity", "sector","occupation")
wage_data[categorial_vars] <- lapply(wage_data[categorial_vars], function(x) as.factor(as.character(x)))

for (var in categorial_vars){
    print(ggplot(data = wage_data, aes_string(x = var, fill = "wage_level" )) +
      geom_bar( position = "identity", alpha = 0.7) +
      labs(title = paste0("Histogramme de la variable ", var , " en fonction du salaire moyen"),
           x = var, y = "Fréquence") +
      scale_fill_manual(values = c("blue", "red")) +  # Couleurs des labels
      theme_minimal())
}

# Variable Education

<img src="image-20250330-163929.png" width="" align="" />

## Vraisemblance

Choix de la vraisemblance :

Étant donné la concentration des données à une valeur spécifique, nous avons choisi une distribution de Poisson pour modéliser le nombre d'années d'études. 

$X_{i,s} | \beta_{education,s} \sim \mathcal{Poisson}(\beta_{education,s} )$ et $s \in \{l,h\}$

## Prior

L'histogramme montre une distribution avec un pic très prononcé à 12 années d'études, indiquant que la plupart des individus ont ce niveau d'éducation.
Il y a quelques valeurs dispersées autour de ce pic, mais la majorité des données est concentrée à 12 ans.


$\beta_{education,s } \sim \mathcal{Gamma}(\alpha, \beta)$ avec $s \in \{l,h\}$

On souhaite que $E(\beta_{education,l}) = 12$ et $V(\beta_{education,l}) = 20$

On souhaite que $E(\beta_{education,h}) = 12$ et $V(\beta_{education,h}) = 20$

Pour la loi gamma $E(\beta) = \frac{\alpha}{\beta}$ et $V(\beta) = \frac{\alpha}{\beta^2}$

donc $\beta_{education,s} \sim \mathcal{Gamma}(7.2, 0.6)$

## Implementation en JAGS

In [185]:
model_string_education <- "
model {

  # Priors
  beta_education_l~ dgamma(7.2, 0.6)
  beta_education_h ~ dgamma(7.2, 0.6)
  
  
  for (i in 1:n1) {
    y1_education[i] ~ dpois(beta_education_l)
  }
  
  for (i in 1:n2) {
    y2_education[i] ~ dpois(beta_education_h)
  }
}
"
education_l <- wage_data$education[wage_data$wage_level == 0]
education_h <- wage_data$education[wage_data$wage_level == 1]

# Définition des données
y1_education <- education_l
y2_education <- education_h
data_list <- list(y1 = y1_education, y2 = y2_education, n1 = length(y1_education), n2 = length(y2_education))

# Compiling and producing posterior samples from the model.
jags_samples_education <- run.jags(model = model_string_education, data = data_list, monitor = c("beta_education_l", "beta_education_h"), n.chains = 3, adapt = 1000, burnin = 1000, thin = 1)

# Plotting and summarizing the posterior distribution
jags_samples_education
plot_credible_intervals(jags_samples_education)

In [181]:
analyze_mcmc_difference <- function(jags_samples, param1, param2) {

    # Conversion des échantillons en data frame
  s <- as.data.frame(jags_samples$mcmc %>% lapply(as_tibble) %>% bind_rows())
  
  # Calcul de la différence
  diff_samples <- s[[param1]] - s[[param2]]
  
  # Calcul de la probabilité
  prob <- mean(diff_samples < 0)
  
  # Graphique
  hist(diff_samples, 
       main = paste("Différence entre", param1, "et", param2),
       xlab = paste(param1, "-", param2),
       col = "skyblue",
       border = "white",
       breaks = 30)
  
  # Légende
  legend("topright", 
         legend = paste0("mean(", param1, " < ", param2, ") = ", round(prob, 3)),
         bty = "n")
  
  # Retour simple
  return(prob)
}

analyze_mcmc_difference(jags_samples_education, "beta_education_l", "beta_education_h")

$ \beta_{education,l} $  : L'effet de l'éducation pour les personnes dont le salaire est inférieur à la moyenne.

$ \beta_{education,h} $  : L'effet de l'éducation pour les personnes dont le salaire est supérieur à la moyenne.

mean($ \beta_{education,l} - \beta_{education,h} < 0$) = 1 : Cela signifie que,l'effet de l'éducation sur la probabilité de se retrouver dans le groupe lower (salaire inférieur à la moyenne) est plus faible que l'effet de l'éducation sur la probabilité de se retrouver dans le groupe higher (salaire supérieur à la moyenne).

## MCMC

In [182]:
# Fonction pour tracer traceplots et autocorrélogrammes
plot_mcmc <- function(jags_samples) {
  
  # Convertir en objet mcmc
  mcmc_samples <- as.mcmc.list(jags_samples)

   # Traceplots (convergence)
  plot(mcmc_samples, trace = TRUE, density = FALSE)
  
  # Tracer l'autocorrélation pour chaque chaîne
  autocorr.plot(jags_samples)
}

# Utilisation de la fonction
plot_mcmc(jags_samples_education)

## Fake data check

In [183]:
generate_fake_data <- function(n1, n2) {
  beta_education_l_fake <- 12  
  beta_education_h_fake <- 14
  
  # Générer des données factices à partir de la distribution de Poisson
  y1_fake <- rpois(n1, beta_education_l_fake)
  y2_fake <- rpois(n2, beta_education_h_fake)
  
  return(list(y1_fake = y1_fake, y2_fake = y2_fake))
}

# Générer des données factices à partir des échantillons postérieurs
fake_data <- generate_fake_data(length(education_l), length(education_h))


data_list <- list(y1 = fake_data$y1_fake, y2 = fake_data$y2_fake, n1 = length(fake_data$y1_fake), n2 = length(fake_data$y2_fake))

summary(fake_data$y1_fake)
summary(fake_data$y2_fake)
summary(y1_education)
summary(y2_education)


# Compiling and producing posterior samples from the model.
jags_samples_education2 <- run.jags(model = model_string_education, data = data_list, monitor = c("beta_education_l", "beta_education_h"))

# Plotting and summarizing the posterior distribution
jags_samples_education2
plot_credible_intervals(jags_samples_education2)

## Conclusion

Les résultats montrent une bonne convergence et une estimation précise des paramètres.

Les intervalles de crédibilité à 95% pour beta_education_l et beta_education_h sont bien définis, sans chevauchement avec zéro, ce qui suggère que les deux paramètres sont significatifs.

Les valeurs de psrf proches de 1 et les petites erreurs MCMC indiquent que les chaînes MCMC ont bien convergé.

L'écart-type est faible, ce qui montre que les estimations des paramètres sont relativement précises.

# Variable South

<img src="image-20250330-164413.png" width="" align="" />

## Vraisemblance

Étant donné que "south" est une variable binaire (Non-South, South), une distribution de Bernoulli est appropriée pour modéliser cette variable. La fonction de vraisemblance pour une variable de Bernoulli est donnée par : 

$X_{i,s} \sim \text{Bernoulli}(\beta_{south,s})$ avec $s \in \{l,h\}$

## Choix du Prior

Pour le prior, une distribution Beta est souvent utilisée comme conjugate prior pour une distribution de Bernoulli. La distribution Beta est définie par :

$\beta_{south,s} \sim \mathcal{Beta}(1, 1)$ avec $s \in \{l,h\}$



## Implémentation en Jags

In [171]:
# Get data for each wage level 
south_l <- wage_data$south[wage_data$wage_level == 0]
south_h <- wage_data$south[wage_data$wage_level == 1]

# Convert to numeric vectors
y1_south <- as.numeric(south_l)
y2_south <- as.numeric(south_h)

# Set up data list
data_list <- list(y1 = y1_south, y2 = y2_south, n1 = length(y1_south), n2 = length(y2_south))

# Define model
model_string_south <- "
model {
  # Priors
  beta_south_l ~ dbeta(1, 1)
  beta_south_h ~ dbeta(1, 1)
  
  # Likelihood
  for (i in 1:n1) {
    y1_south[i] ~ dbern(beta_south_l)
  }
  
  for (i in 1:n2) {
    y2_south[i] ~ dbern(beta_south_h)
  }
}
"

# Run JAGS model with more iterations and samples
jags_samples_south <- run.jags(
  model = model_string_south,
  data = data_list,
  monitor = c("beta_south_l", "beta_south_h"),
  n.chains = 3,
  adapt = 5000,
  burnin = 5000,
  sample = 10000,
  thin = 10
)

# Print summary
print(jags_samples_south)
# Plot credible intervals
plot_credible_intervals(jags_samples_south)

In [172]:
analyze_mcmc_difference(jags_samples_south, "beta_south_l", "beta_south_h")

Il est difficile de démontrer que la localisation géographique dans le sud a un impact distinct et fort sur les salaires bas ou élevés, car les valeurs des betas sont proches de zéro, ce qui suggère une faible relation. Les intervalles de crédibilité incluant 0 confirment que l'effet de la région Sud sur le salaire n'est pas statistiquement significatif.

## MCMC 

In [173]:
plot_mcmc(jags_samples_south)

Les résultats des modèles sur les données réelles et les données fictives sont similaires. Les deux ensembles montrent une bonne convergence (avec des valeurs PSRF proches de 1)

## Fake data check

In [174]:
generate_fake_data <- function(n1, n2) {
  beta_south_l_fake <- 0.5
  beta_south_h_fake <- 0.5
  
  # Générer des données factices à partir de la distribution de bernouilli
  # Convert logical to numeric (0/1) values
  y1_fake <- as.numeric(rbernoulli(n1, beta_south_l_fake))
  y2_fake <- as.numeric(rbernoulli(n2, beta_south_h_fake))
  
  return(list(y1_fake = y1_fake, y2_fake = y2_fake))
}

# Générer des données factices à partir des échantillons postérieurs
fake_data <- generate_fake_data(length(south_l), length(south_h))

# Comparer les données factices avec les données réelles
summary(fake_data$y1_fake)
summary(fake_data$y2_fake)
summary(y1_south)
summary(y2_south)

data_list <- list(y1 = fake_data$y1_fake, 
                 y2 = fake_data$y2_fake, 
                 n1 = length(fake_data$y1_fake), 
                 n2 = length(fake_data$y2_fake))

# Compiling and producing posterior samples from the model.
jags_samples_south2 <- run.jags(model = model_string_south,  # Fixed model string name
                              data = data_list, 
                              monitor = c("beta_south_l", "beta_south_h"),
                              n.chains = 3,
                              adapt = 5000,
                              burnin = 5000,
                              sample = 10000,
                              thin = 10)

# Plotting and summarizing the posterior distribution
print(jags_samples_south2)
plot_credible_intervals(jags_samples_south2)

## Conclusion

* Convergence MCMC : les chaînes semblent osciller autour d'une valeur centrale, ce qui est un bon signe de convergence. elles sont bien mélangées et ne montrent pas de tendance systématique, ce qui est un autre indcateur positif de convergence.
Cela suggère que les résultats du modèle peuvent être considérés comme fiables (les distributions postérieurs des coefficients beta_south_l et beta_south_h sont fiables)
* Fake Data Check : Les médianes et moyennes des paramètres beta_south_1 et beta_south_h sont très proches entre les données réelles et fictives, autour de 0.5.Cela suggère que le modèle se comporte de manière similaire dans les deux cas. Les intervalles crédibles à 95% sont larges et se chevauchent largement, ce qui indique une incertitude importante dans l'estimation des paramètres.

# Variable unionmember

<img src="image-20250330-174853.png" width="" align="" />

## Vraisemblance

Etant donné que "unionmember" est une variable binaire (Non-Member, Member), une distribution de Bernoulli est appropriée pour modéliser cette variable avec des priors de loi beta (1,1). 
$$
X_{i,s} \mid \beta_{union,s}\sim B(\beta_{union,s}) \\ 
$$
avec $s \in \{l,h\}$

## Choix du prior

priors :
$$ 
\beta_{union,s} ∼ Beta(1,1)
$$

avec $s \in \{l,h\}$

## Implémentation en Jags

In [175]:
# Get data for each wage level 
union_l <- wage_data$unionmember[wage_data$wage_level == 0]
union_h <- wage_data$unionmember[wage_data$wage_level == 1]

# Convert to numeric vectors
y1_union <- as.numeric(union_l)
y2_union <- as.numeric(union_h)

# Set up data list
data_list <- list(y1 = y1_union, y2 = y2_union, n1 = length(y1_union), n2 = length(y2_union))

# Define model
model_string_union <- "
model {
  # Priors
  beta_union_l ~ dbeta(1, 1)
  beta_union_h ~ dbeta(1, 1)
  
  # Likelihood
  for (i in 1:n1) {
    y1_union[i] ~ dbern(beta_union_l)
  }
  
  for (i in 1:n2) {
    y2_union[i] ~ dbern(beta_union_h)
  }
}
"

# Run JAGS model with more iterations and samples
jags_samples_union <- run.jags(
  model = model_string_union,
  data = data_list,
  monitor = c("beta_union_l", "beta_union_h"),
  n.chains = 3,
  adapt = 5000,
  burnin = 5000,
  sample = 10000,
  thin = 10
)

# Print summary
print(jags_samples_union)
# Plot credible intervals
plot_credible_intervals(jags_samples_union)


In [176]:
analyze_mcmc_difference(jags_samples_union, "beta_union_l", "beta_union_h")

De même que pour la variable south, il est difficile de démontrer que l'appartenance à un syndicat a un impact distinct et fort sur les salaires bas ou élevés, car les valeurs des betas sont proches de zéro, ce qui suggère une faible relation. Les intervalles de crédibilité incluant 0 renforcent cette conclusion

## MCMC

In [184]:
plot_mcmc(jags_samples_union) 

## Fake Check Data

In [178]:
generate_fake_data <- function(n1, n2) {
  beta_union_l_fake <- 0.5
  beta_union_h_fake <- 0.5
  
  # Générer des données factices à partir de la distribution de bernouilli
  # Convert logical to numeric (0/1) values
  y1_fake <- as.numeric(rbernoulli(n1, beta_union_l_fake))
  y2_fake <- as.numeric(rbernoulli(n2, beta_union_h_fake))
  
  return(list(y1_fake = y1_fake, y2_fake = y2_fake))
}

# Générer des données factices à partir des échantillons postérieurs
fake_data <- generate_fake_data(length(union_l), length(union_h))

# Comparer les données factices avec les données réelles
summary(fake_data$y1_fake)
summary(fake_data$y2_fake)
summary(y1_union)
summary(y2_union)

data_list <- list(y1 = fake_data$y1_fake, 
                 y2 = fake_data$y2_fake, 
                 n1 = length(fake_data$y1_fake), 
                 n2 = length(fake_data$y2_fake))

# Compiling and producing posterior samples from the model.
jags_samples_union2 <- run.jags(model = model_string_union,  # Fixed model string name
                              data = data_list, 
                              monitor = c("beta_union_l", "beta_union_h"),
                              n.chains = 3,
                              adapt = 5000,
                              burnin = 5000,
                              sample = 10000,
                              thin = 10)

# Plotting and summarizing the posterior distribution
print(jags_samples_union2)
# Plot credible_interval
plot_credible_intervals(jags_samples_union2)

# Conclusion

* Intervalle crédible :  les IC sont extrêmement larges, ce qui indique une incertitude importante sur l'estimation de l'effet.
* Convergence MCMC : les trajectoires de chaines se chevauchent et restent stationnaires après un certain nombre d'itérations, cela suggère que l'espace des paramètres a été correctement exploré et que les estimations postérieures sont fiables.
* Fake Check Data : Les médianes et moyennes des paramètres beta_union_l et beta_union_h sont très proches de 0.5 dans tous les cas, ce qui suggère que le modèle attribue un effet neutre (ni positif ni négatif) au facteur "syndicat".

 

# Variable Age

On observe que l'age des individus est fortement correle avec l'experience professionnelle, ce qui fait sens puisque plus l'age est grand, plus l'experience professionnelle est longue. Pour éviter une instabilité des estimations, nous supprimons écartons la variable Age.

In [116]:
ggplot(wage_data, aes(x=age, y=workexp)) +
  geom_point(alpha=0.5, color="purple") +
  labs(title="Relation entre Age et Expérience Professionnelle", x="Age", y="Expérieence Professionnelle")

# Variable Work experience

<img src="image-20250330-163826.png" width="" align="" />

## Vraisemblance

Une distribution normale semble appropriée pour modéliser les années d'expérience au travail, car les données suivent une forme de cloche.

$X_{i,s} | \beta_{workexp,s} \sim \mathcal{Normal}(\beta_{workexp,s})$

Avec $\beta_{workexp,s} = (\mu_s, \sigma_s)$ et $ s \in \{l,h\}$

## Choix du prior

$\mu_s \sim \mathcal{Uniform}(0, 30)$ 
$\sigma_s \sim \mathcal{Uniform}(0, 50)$

## Implémentation en JAGS

In [186]:
model_string_workexp <- "
model {

  mu_l ~ dunif(0, 30)
  sigma_l ~ dunif(0, 50)
  precision_l <- 1/sigma_l^2

  mu_h ~ dunif(0, 30)
  sigma_h ~ dunif(0, 50)
  precision_h <- 1/sigma_h^2
  
  for (i in 1:n1) {
    y1_workexp[i] ~ dnorm(mu_l, precision_l)
  }
  
  for (i in 1:n2) {
    y2_workexp[i] ~ dnorm(mu_h, precision_h)
  }

  
}
"

# Définition des données
y1_workexp <- wage_data$workexp[wage_data$wage_level == 0]
y2_workexp <- wage_data$workexp[wage_data$wage_level == 1]
data_list <- list(y1 = y1_workexp, y2 = y2_workexp, n1 = length(y1_workexp), n2 = length(y2_workexp))



# Compiling and producing posterior samples from the model.
jags_samples_workexp <- run.jags(model = model_string_workexp, data = data_list, monitor = c("mu_l", "sigma_l", "mu_h", "sigma_h"), n.chains = 3, adapt = 1000, burnin = 1000, thin = 1)

# Plotting and summarizing the posterior distribution
jags_samples_workexp
plot_credible_intervals(jags_samples_workexp)

In [187]:
analyze_mcmc_difference(jags_samples_workexp, "mu_l", "mu_h")

Les individus ayant un salaire inférieur à la moyenne ont en moyenne plus d'expérience professionnelle que ceux ayant un salaire supérieur à la moyenne.

## MCMC

In [188]:
plot_mcmc(jags_samples_workexp)

## Fake data check

In [189]:
generate_fake_data <- function(n1, n2) {
    mu_l_fake <- 15
    sigma_l_fake <- 25
    mu_h_fake <- 15
    sigma_h_fake <- 24
  
  # Générer des données factices à partir de la distribution de Poisson
  y1_fake <- rnorm(n1, mu_l_fake, sigma_l_fake)
  y2_fake <- rnorm(n2, mu_h_fake, sigma_h_fake)
  
  return(list(y1_fake = y1_fake, y2_fake = y2_fake))
}

# Générer des données factices à partir des échantillons postérieurs
fake_data <- generate_fake_data(length(y1_workexp), length(y2_workexp))

# Comparer les données factices avec les données réelles
summary(fake_data$y1_fake)
summary(fake_data$y2_fake)
summary(y1_workexp)
summary(y2_workexp)

data_list <- list(y1 = fake_data$y1_fake, y2 = fake_data$y2_fake, n1 = length(fake_data$y1_fake), n2 = length(fake_data$y2_fake))


# Compiling and producing posterior samples from the model.
jags_samples_workexp2 <- run.jags(model = model_string_workexp, data = data_list, monitor = c("mu_l", "sigma_l", "mu_h", "sigma_h"))

# Plotting and summarizing the posterior distribution
jags_samples_workexp2
plot_credible_intervals(jags_samples_workexp2)

# Variable female

<img src="image-20250330-165243.png" width="" align="" />

## Vraisemblance

Chaque observation suit une loi de Bernoulli :

$X_{i,s} \sim \text{Bernoulli}(\beta_{female,s})$

avec $ s \in \{l,h\}$


## Choix du Prior

$\beta_{female,l} \sim \mathcal{Beta}(173, 151)$
$\beta_{female,h} \sim \mathcal{Beta}(74, 134)$

## Implémentation en JAGS

In [197]:
# Get data for each wage level 
female_l <- wage_data$female[wage_data$wage_level == 0]
female_h <- wage_data$female[wage_data$wage_level == 1]

# Convert to numeric vectors
y1_female <- as.numeric(female_l)
y2_female <- as.numeric(female_h)

# Set up data list
data_list <- list(y1 = y1_female, y2 = y2_female, n1 = length(y1_female), n2 = length(y2_female))

# Define model
model_string_female <- "
model {
  # Priors
  beta_female_l ~ dbeta(173, 151)
  beta_female_h ~ dbeta(74, 134)
  
  # Likelihood
  for (i in 1:n1) {
    y1_female[i] ~ dbern(beta_female_l)
  }
  
  for (i in 1:n2) {
    y2_female[i] ~ dbern(beta_female_h)
  }
}
"

# Run JAGS model with more iterations and samples
jags_samples_female <- run.jags(
  model = model_string_female,
  data = data_list,
  monitor = c("beta_female_l", "beta_female_h"),
  n.chains = 3,
  adapt = 5000,
  burnin = 5000,
  sample = 10000,
  thin = 10
)

# Print summary
print(jags_samples_female)
# Plot credible intervals
plot_credible_intervals(jags_samples_female)

In [198]:
analyze_mcmc_difference(jags_samples_female, "beta_female_l", "beta_female_h")

L'effet d'être une femme est plus fort pour les bas salaires que pour les hauts salaires.

## MCMC

In [199]:
plot_mcmc(jags_samples_female)

## Fake data check

In [200]:
generate_fake_data <- function(n1, n2) {
  beta_female_l_fake <- 0.5
  beta_female_h_fake <- 0.5
  
  # Générer des données factices à partir de la distribution de bernouilli
  # Convert logical to numeric (0/1) values
  y1_fake <- as.numeric(rbernoulli(n1, beta_female_l_fake))
  y2_fake <- as.numeric(rbernoulli(n2, beta_female_h_fake))
  
  return(list(y1_fake = y1_fake, y2_fake = y2_fake))
}

# Générer des données factices à partir des échantillons postérieurs
fake_data <- generate_fake_data(length(female_l), length(female_h))

# Comparer les données factices avec les données réelles
summary(fake_data$y1_fake)
summary(fake_data$y2_fake)
summary(y1_female)
summary(y2_female)

data_list <- list(y1 = fake_data$y1_fake, 
                 y2 = fake_data$y2_fake, 
                 n1 = length(fake_data$y1_fake), 
                 n2 = length(fake_data$y2_fake))

# Compiling and producing posterior samples from the model.
jags_samples_female2 <- run.jags(model = model_string_female,  # Fixed model string name
                              data = data_list, 
                              monitor = c("beta_female_l", "beta_female_h"),
                              n.chains = 3,
                              adapt = 5000,
                              burnin = 5000,
                              sample = 10000,
                              thin = 10)

# Plotting and summarizing the posterior distribution
print(jags_samples_female2)
plot_credible_intervals(jags_samples_female2)

## Conclusion

### Interprétation

- 🔹 **`beta_female_l`** : Le modèle estime à **53.5 %** la probabilité que la femme classée comme ayant un **salaire faible** soit effectivement dans ce cas.
- 🔹 **`beta_female_h`** : La probabilité estimée d’un **salaire élevé** pour la deuxième femme est de **35.9 %**, ce qui reflète une **plus grande incertitude**.

### Convergence

- Tous les diagnostics sont bons :
  - MC%ofSD < 1 %
  - PSRF proche de 1
  - Taille effective > 14 000

Cela indique que les chaînes MCMC ont bien convergé.

# Variable Married

<img src="image-20250330-164951.png" width="" align="" />

## Vraisemblance


Etant donné que "married" est une variable binaire (Single, Married), une distribution de Bernoulli est appropriée pour modéliser cette variable. La fonction de vraisemblance pour une variable de Bernoulli est donnée par :

$X_{i,s} \sim \text{Bernoulli}(\beta_{married,s})$ avec $s \in \{l,h\}$



## Choix du Prior

En ce qui concerne le choix du prior, la distribution Beta est couramment utilisée comme prior conjugué pour une distribution de Bernoulli.

Pour le choix des paramètres du prior et à partir de l’histogramme, on considère :

$\beta_{married,l} \sim \mathcal{Beta}(191, 131)$ 
$\beta_{married,l} \sim \mathcal{Beta}(161, 61)$ 

## Implémentation en JAGS

In [204]:
# ------------------------------
# 1. Définir le modèle JAGS (avec priors informatifs)
# ------------------------------
model_string_married <- "
model {

  # Priors inspirés des données observées
  beta_married_l ~ dbeta(191, 131)  # pour salaire < moyenne
  beta_married_h ~ dbeta(161, 61)   # pour salaire ≥ moyenne

  # Vraisemblance
  for (i in 1:n1) {
    y1_married[i] ~ dbern(beta_married_l)
  }

  for (i in 1:n2) {
    y2_married[i] ~ dbern(beta_married_h)
  }
} 
"

# ------------------------------
# 2. Préparer les données
# ------------------------------
# Convertir la variable 'married' en binaire (1 = Married, 0 = Single)
wage_data$married <- as.numeric(wage_data$married == "Married")

# Séparer selon le niveau de salaire
married_l <- wage_data$married[wage_data$wage_level == 0]  # faible salaire
married_h <- wage_data$married[wage_data$wage_level == 1]  # haut salaire

# Convertir en vecteur numérique
y1_married <- as.numeric(married_l)
y2_married <- as.numeric(married_h)

# Créer la liste des données
data_list_married <- list(
  y1 = y1_married,
  y2 = y2_married,
  n1 = length(married_l),
  n2 = length(married_h)
)

# ------------------------------
# 3. Exécuter le modèle
# ------------------------------

jags_samples_married <- run.jags(
  model = model_string_married,
  data = data_list_married,
  monitor = c("beta_married_l", "beta_married_h"),
  n.chains = 3,
  adapt = 1000,
  burnin = 1000,
  sample = 5000,
  thin = 1
)


# ------------------------------
# 4. Résultats
# ------------------------------
jags_samples_married
plot_credible_intervals(jags_samples_married)



In [206]:
analyze_mcmc_difference(jags_samples_married, "beta_married_l", "beta_married_h")

être marié est fortement associé à un salaire plus élevé dans le modèle.

## MCMC

In [202]:
plot_mcmc(jags_samples_married)

## Fake data check

In [205]:
# ------------------------------
# 1. Générer des données factices de statut marital par niveau de salaire
# ------------------------------
generate_fake_married_data <- function(n_low, n_high, p_low = 0.59, p_high = 0.73) {
  # Générer pour les faibles salaires
  y1 <- rbinom(n_low, 1, p_low)

  # Générer pour les hauts salaires
  y2 <- rbinom(n_high, 1, p_high)

  return(list(y1 = y1, y2 = y2))
}


# ------------------------------
# 2. Generate fake data
# ------------------------------

n1 <- length(married_l)
n2 <- length(married_h)

set.seed(123)
fake <- generate_fake_married_data(n1, n2)

data_list_fake_married <- list(
  y1 = fake$y1,
  y2 = fake$y2,
  n1 = n1,
  n2 = n2
)

# Comparer les données factices avec les données réelles
summary(fake$y1)
summary(fake$y2)
summary(y1_married)
summary(y2_married)

# ------------------------------
# 3. JAGS model (adjusted for married)
# ------------------------------
model_string_married <- "
model {
  # Priors basés sur l'histogramme
  beta_married_l ~ dbeta(191, 131)
  beta_married_h ~ dbeta(161, 61)

  # Vraisemblance Bernoulli pour chaque groupe
  for (i in 1:n1) {
    y1[i] ~ dbern(beta_married_l)
  }

  for (i in 1:n2) {
    y2[i] ~ dbern(beta_married_h)
  }
}
"


# ------------------------------
# 4. Run the model
# ------------------------------

jags_samples_married2 <- run.jags(
  model = model_string_married,
  data = data_list_fake_married,
  monitor = c("beta_married_l", "beta_married_h"),
  n.chains = 3,
  adapt = 1000,
  burnin = 1000,
  sample = 5000
)


# ------------------------------
# 5. Examine results
# ------------------------------
jags_samples_married2
plot_credible_intervals(jags_samples_married2)


## Conclusion

### Interprétation

- Parmi les individus ayant un **salaire inférieur à la moyenne**, la probabilité d’être marié est estimée à **59,5 %**, avec un intervalle de crédibilité à 95 % entre **55,9 % et 63,4 %**.
  
- Pour les individus ayant un **salaire supérieur ou égal à la moyenne**, cette probabilité est plus élevée : **73,3 %**, avec un intervalle de crédibilité entre **69,3 % et 77,5 %**.

---

### Convergence MCMC

- Tous les diagnostics sont bons :
  - MC%ofSD < 1 %
  - PSRF ≈ 1
  - Taille effective > 14 000

Les chaînes ont bien convergé : les résultats sont fiables et interprétables.

# Variable ethnicity

<img src="image-20250330-170259.png" width="" align="" />

## Vraisemblance

$$
X_{k,s}|\beta_{k,s} \sim \text{Poisson}(\beta_{k,s})
$$

Avec k représente la catégorie de ethnicity (individu White, individu Other, individu Hisp).

## Choix du prior


$$
\begin{align*}
\beta_{\text{White}, \ell} &\sim \text{Gamma}(9, 0.1) \\
\beta_{\text{White}, h} &\sim \text{Gamma}(18, 0.1) \\
\\
\beta_{\text{Other}, \ell} &\sim \text{Gamma}(4.5, 0.1) \\
\beta_{\text{Other}, h} &\sim \text{Gamma}(3, 0.1) \\
\\
\beta_{\text{Hispanic}, \ell} &\sim \text{Gamma}(2.5, 0.1) \\
\beta_{\text{Hispanic}, h} &\sim \text{Gamma}(1, 0.1)
\end{align*}
$$



## Implémentation en JAGS

In [211]:
# ------------------------------
# 1. Préparer les données (agrégation par ethnie)
# ------------------------------
# Liste des groupes ethniques
eth_groups <- c("White", "Other", "Hispanic")

# Comptes par groupe ethnique et niveau de salaire
counts_l <- sapply(eth_groups, function(e) sum(wage_data$wage_level == 0 & wage_data$ethnicity == e))
counts_h <- sapply(eth_groups, function(e) sum(wage_data$wage_level == 1 & wage_data$ethnicity == e))

# Liste des données à passer à JAGS
data_list_ethnicity <- list(
  y_l = counts_l,
  y_h = counts_h
)
# ------------------------------
# 2. Définir le modèle JAGS avec noms explicites
# ------------------------------
model_string_ethnicity <- "
model {
  # Vraisemblance
  y_l_white    ~ dpois(beta_white_l)
  y_h_white    ~ dpois(beta_white_h)

  y_l_other    ~ dpois(beta_other_l)
  y_h_other    ~ dpois(beta_other_h)

  y_l_hispanic ~ dpois(beta_hispanic_l)
  y_h_hispanic ~ dpois(beta_hispanic_h)

  # Priors
  beta_white_l    ~ dgamma(9, 0.1)
  beta_white_h    ~ dgamma(18, 0.1)

  beta_other_l    ~ dgamma(4.5, 0.1)
  beta_other_h    ~ dgamma(3, 0.1)

  beta_hispanic_l ~ dgamma(2.5, 0.1)
  beta_hispanic_h ~ dgamma(1, 0.1)
}
"

# Adapter les noms pour coller à ceux du modèle
data_list_ethnicity_named <- list(
  y_l_white    = counts_l["White"],
  y_h_white    = counts_h["White"],
  y_l_other    = counts_l["Other"],
  y_h_other    = counts_h["Other"],
  y_l_hispanic = counts_l["Hispanic"],
  y_h_hispanic = counts_h["Hispanic"]
)
# ------------------------------
# 3. Exécuter le modèle
# ------------------------------

library(runjags)

jags_samples_ethnicity <- run.jags(
  model = model_string_ethnicity,
  data = data_list_ethnicity_named,
  monitor = c(
    "beta_white_l", "beta_white_h",
    "beta_other_l", "beta_other_h",
    "beta_hispanic_l", "beta_hispanic_h"
  ),
  n.chains = 3,
  adapt = 1000,
  burnin = 1000,
  sample = 5000
)

# ------------------------------
# 4. Résultats
# ------------------------------
print(jags_samples_ethnicity)
plot_credible_intervals(jags_samples_ethnicity)

In [217]:
analyze_mcmc_difference(jags_samples_ethnicity,"beta_white_l","beta_white_h")
analyze_mcmc_difference(jags_samples_ethnicity,"beta_other_l","beta_other_h")
analyze_mcmc_difference(jags_samples_ethnicity,"beta_hispanic_l","beta_hispanic_h")

Les minorités ethniques sont significativement désavantagées en termes d'accès aux hauts salaires. Ce modèle met en évidence une forte inégalité salariale selon l’ethnicité, avec un effet particulièrement marqué pour les Hispaniques et autres minorités non blanches.

## MCMC

In [212]:
plot_mcmc(jags_samples_ethnicity)

## Fake data check

In [214]:
# ------------------------------
# 1. Generate fake Poisson wage counts by ethnicity and wage level
# ------------------------------
generate_fake_poisson_data <- function() {
  # Expected beta values (based on visual estimates)
  beta_white_l <- 90
  beta_white_h <- 180
  beta_other_l <- 45
  beta_other_h <- 30
  beta_hispanic_l <- 25
  beta_hispanic_h <- 10

  # Simulate counts from Poisson
  y_l_white    <- rpois(1, beta_white_l)
  y_h_white    <- rpois(1, beta_white_h)
  y_l_other    <- rpois(1, beta_other_l)
  y_h_other    <- rpois(1, beta_other_h)
  y_l_hispanic <- rpois(1, beta_hispanic_l)
  y_h_hispanic <- rpois(1, beta_hispanic_h)

  return(list(
    y_l_white = y_l_white,
    y_h_white = y_h_white,
    y_l_other = y_l_other,
    y_h_other = y_h_other,
    y_l_hispanic = y_l_hispanic,
    y_h_hispanic = y_h_hispanic
  ))
}

set.seed(123)
fake <- generate_fake_poisson_data()

data_list_fake <- list(
  y_l_white    = fake$y_l_white,
  y_h_white    = fake$y_h_white,
  y_l_other    = fake$y_l_other,
  y_h_other    = fake$y_h_other,
  y_l_hispanic = fake$y_l_hispanic,
  y_h_hispanic = fake$y_h_hispanic
)

model_string_ethnicity <- "
model {
  # Likelihood
  y_l_white    ~ dpois(beta_white_l)
  y_h_white    ~ dpois(beta_white_h)

  y_l_other    ~ dpois(beta_other_l)
  y_h_other    ~ dpois(beta_other_h)

  y_l_hispanic ~ dpois(beta_hispanic_l)
  y_h_hispanic ~ dpois(beta_hispanic_h)

  # Priors based on previous visual inspection
  beta_white_l    ~ dgamma(9, 0.1)
  beta_white_h    ~ dgamma(18, 0.1)
  
  beta_other_l    ~ dgamma(4.5, 0.1)
  beta_other_h    ~ dgamma(3, 0.1)

  beta_hispanic_l ~ dgamma(2.5, 0.1)
  beta_hispanic_h ~ dgamma(1, 0.1)
}
"

jags_samples_ethnicity_fake <- run.jags(
  model = model_string_ethnicity,
  data = data_list_fake,
  monitor = c(
    "beta_white_l", "beta_white_h",
    "beta_other_l", "beta_other_h",
    "beta_hispanic_l", "beta_hispanic_h"
  ),
  n.chains = 3,
  adapt = 1000,
  burnin = 1000,
  sample = 5000
)

print(jags_samples_ethnicity_fake)
plot_credible_intervals(jags_samples_ethnicity_fake)
summary(jags_samples_ethnicity_fake)

## Conclusion

Ce tableau compare la valeur moyenne estimée de \( \beta \), c’est-à-dire **le nombre moyen d’individus par groupe ethnique** selon le niveau de salaire.

| Groupe ethnique | Niveau salaire | \( \beta\) réel | \( \beta \) simulé | Écart absolu |
|----------------|---------------|----------------|----------------|--------------|
| White         | < Moyenne     | 239.14         | 84.52          |   154.62     |
| White         | ≥ Moyenne     | 185.35         | 193.52         |   8.17       |
| Other         | < Moyenne     | 46.80          | 34.08          |   12.72      |
| Other         | ≥ Moyenne     | 20.95          | 30.00          |   9.05       |
| Hispanic      | < Moyenne     | 22.33          | 32.19          |   9.86       |
| Hispanic      | ≥ Moyenne     | 5.44           | 10.87          |   5.43       |

---

### Interprétation

- **beta_white_h** (White, salaire élevé) est **bien capturé** par les données simulées : la moyenne simulée (193.52) est proche de la valeur réelle (185.35).

- **beta_white_l** (White, salaire faible) est **largement sous-estimé** dans les données factices (~84 vs 239 réels). Cela montre que les données simulées **ne reflètent pas bien la distribution réelle** des salaires faibles chez les individus blancs.

- **beta_hispanic_h** et **beta_other_h** sont


# Variable sector

<img src="image-20250330-170309.png" width="" align="" />

## Vraisemblance

$$
X_{k, s} | \beta_{k, s} \sim \text{Poisson}(\beta_{k, s})
$$

avec $ s \in \{ \ell, h \} $

## Choix du prior

$$
\begin{aligned}
\beta_{\text{Construction}, \ell} &\sim \text{Gamma}(1, 0.1) & \quad \beta_{\text{Construction}, h} &\sim \text{Gamma}(1, 0.1) \\
\beta_{\text{Manufacturing}, \ell} &\sim \text{Gamma}(3, 0.1) & \quad \beta_{\text{Manufacturing}, h} &\sim \text{Gamma}(3, 0.1) \\
\beta_{\text{Other}, \ell} &\sim \text{Gamma}(13, 0.1) & \quad \beta_{\text{Other}, h} &\sim \text{Gamma}(15, 0.1)
\end{aligned}
$$

## Implémentation en JAGS

In [221]:
# Compter les individus par secteur et niveau de salaire
counts_l <- table(wage_data$sector[wage_data$wage_level == 0])
counts_h <- table(wage_data$sector[wage_data$wage_level == 1])

# Préparer les données pour JAGS
data_list_sector <- list(
  y_l_Construction   = as.numeric(counts_l["Construction"]),
  y_h_Construction   = as.numeric(counts_h["Construction"]),
  y_l_Manufacturing  = as.numeric(counts_l["Manufacturing"]),
  y_h_Manufacturing  = as.numeric(counts_h["Manufacturing"]),
  y_l_Other          = as.numeric(counts_l["Other"]),
  y_h_Other          = as.numeric(counts_h["Other"])
)
model_string_sector <- "
model {
  # Likelihood
  y_l_Construction   ~ dpois(beta_l_Construction)
  y_h_Construction   ~ dpois(beta_h_Construction)

  y_l_Manufacturing  ~ dpois(beta_l_Manufacturing)
  y_h_Manufacturing  ~ dpois(beta_h_Manufacturing)

  y_l_Other          ~ dpois(beta_l_Other)
  y_h_Other          ~ dpois(beta_h_Other)

  # Priors (adjusted from histogram)
  beta_l_Construction   ~ dgamma(1, 0.1)
  beta_h_Construction   ~ dgamma(1, 0.1)

  beta_l_Manufacturing  ~ dgamma(3, 0.1)
  beta_h_Manufacturing  ~ dgamma(3, 0.1)

  beta_l_Other          ~ dgamma(13, 0.1)
  beta_h_Other          ~ dgamma(15, 0.1)
}
"
library(runjags)

jags_samples_sector <- run.jags(
  model = model_string_sector,
  data = data_list_sector,
  monitor = c(
    "beta_l_Construction", "beta_h_Construction",
    "beta_l_Manufacturing", "beta_h_Manufacturing",
    "beta_l_Other", "beta_h_Other"
  ),
  n.chains = 3,
  adapt = 1000,
  burnin = 1000,
  sample = 5000
)
print(jags_samples_sector)
plot_credible_intervals(jags_samples_sector)
summary(jags_samples_sector)


In [223]:
analyze_mcmc_difference(jags_samples_sector,'beta_l_Construction','beta_h_Construction')

analyze_mcmc_difference(jags_samples_sector,'beta_l_Manufacturing','beta_h_Manufacturing')

analyze_mcmc_difference(jags_samples_sector,'beta_l_Other','beta_h_Other')

Le secteur de la construction offre une meilleure répartition des salaires, avec une proportion notable de hauts salaires.

Le secteur manufacturier semble relativement équilibré.

Les autres secteurs (services, agriculture, etc.) présentent une forte concentration de bas salaires, ce qui suggère une précarité plus importante.

## MCMC

In [224]:
plot_mcmc(jags_samples_sector)

## Fake data check

In [226]:
generate_fake_sector_data <- function() {
  list(
    y_l_Construction   = rpois(1, 10),
    y_h_Construction   = rpois(1, 10),
    y_l_Manufacturing  = rpois(1, 30),
    y_h_Manufacturing  = rpois(1, 30),
    y_l_Other          = rpois(1, 130),
    y_h_Other          = rpois(1, 150)
  )
}


set.seed(123)
fake_sector <- generate_fake_sector_data()

data_list_sector_fake <- fake_sector
jags_samples_sector_fake <- run.jags(
  model = model_string_sector,
  data = data_list_sector_fake,
  monitor = c(
    "beta_l_Construction", "beta_h_Construction",
    "beta_l_Manufacturing", "beta_h_Manufacturing",
    "beta_l_Other", "beta_h_Other"
  ),
  n.chains = 3,
  adapt = 1000,
  burnin = 1000,
  sample = 5000
)
print(jags_samples_sector_fake)
plot_credible_intervals(jags_samples_sector_fake)
summary(jags_samples_sector_fake)


## Conclusion

Ce tableau compare les valeurs moyennes estimées de \( \lambda \) obtenues avec les données réelles et les données simulées.

| Secteur        | Salaire       | \( \lambda \) réel | \( \lambda \) simulé | Écart absolu |
|---------------|--------------|--------------------|----------------------|--------------|
| Construction   | < Moyenne     | 10.91              | 8.20                 | 2.71         |
| Construction   | ≥ Moyenne     | 12.76              | 9.08                 | 3.68         |
| Manufacturing  | < Moyenne     | 48.18              | 37.17                | 11.01        |
| Manufacturing  | ≥ Moyenne     | 47.27              | 29.98                | 17.29        |
| Other         | < Moyenne     | 250.01             | 131.07               |   118.94     |
| Other         | ≥ Moyenne     | 148.97             | 169.02               |   20.05      |

---

### Analyse

- Les valeurs simulées sont **cohérentes** pour le secteur **Construction**, bien qu'un léger écart subsiste.
- Pour **Manufacturing**, les données simulées **sous-estiment** de manière notable les comptes réels, surtout pour les hauts salaires.
- Dans le cas du secteur **Other**, les \( \lambda \) simulés **sous-estiment fortement** les bas salaires (131 vs 250), mais surestiment légèrement les hauts salaires.
- Toutefois, le modèle reproduit **globalement bien les tendances** observées.

---

### Conclusion

Les **priors utilisés sont raisonnables**, mais pour améliorer la précision :
- On peut **ajuster les hyperparamètres Gamma** pour mieux capturer les distributions réelles, notamment dans le secteur "Other".
- Une **meilleure prise en compte de l'hétérogénéité** des secteurs améliorerait encore la modélisation.


# Variable occupation

<img src="image-20250330-170318.png" width="" align="" />

## Vraisemblance

$$
X_{k, \ell} | \beta_{k, s} \sim \text{Poisson}(\beta_{k, s})
$$

Avec $ s \in \{ \ell, h \}$

## Choix du prior

$$
\begin{aligned}
\beta_{\text{Clerical}, \ell} &\sim \text{Gamma}(4, 0.1) & \quad \beta_{\text{Clerical}, h} &\sim \text{Gamma}(3, 0.1) \\
\beta_{\text{Management}, \ell} &\sim \text{Gamma}(2, 0.1) & \quad \beta_{\text{Management}, h} &\sim \text{Gamma}(3, 0.1) \\
\beta_{\text{Other}, \ell} &\sim \text{Gamma}(4, 0.1) & \quad \beta_{\text{Other}, h} &\sim \text{Gamma}(6, 0.1) \\
\beta_{\text{Professional}, \ell} &\sim \text{Gamma}(3.5, 0.1) & \quad \beta_{\text{Professional}, h} &\sim \text{Gamma}(3.5, 0.1) \\
\beta_{\text{Sales}, \ell} &\sim \text{Gamma}(1.5, 0.1) & \quad \beta_{\text{Sales}, h} &\sim \text{Gamma}(1, 0.1) \\
\beta_{\text{Service}, \ell} &\sim \text{Gamma}(3.5, 0.1) & \quad \beta_{\text{Service}, h} &\sim \text{Gamma}(1.5, 0.1)
\end{aligned}
$$

## Implémentation en JAGS

In [227]:
# Liste des groupes occupation
occupation_groups <- unique(wage_data$occupation)

# Comptes par groupe et niveau de salaire
counts_l <- sapply(occupation_groups, function(g) sum(wage_data$wage_level == 0 & wage_data$occupation == g))
counts_h <- sapply(occupation_groups, function(g) sum(wage_data$wage_level == 1 & wage_data$occupation == g))

# Créer la liste des données nommées
data_list_occupation <- list(
  y_l_Clerical     = counts_l["Clerical"],
  y_h_Clerical     = counts_h["Clerical"],
  y_l_Management   = counts_l["Management"],
  y_h_Management   = counts_h["Management"],
  y_l_Other        = counts_l["Other"],
  y_h_Other        = counts_h["Other"],
  y_l_Professional = counts_l["Professional"],
  y_h_Professional = counts_h["Professional"],
  y_l_Sales        = counts_l["Sales"],
  y_h_Sales        = counts_h["Sales"],
  y_l_Service      = counts_l["Service"],
  y_h_Service      = counts_h["Service"]
)

model_string_occupation <- "
model {
  # Likelihoods
  y_l_Clerical     ~ dpois(beta_l_Clerical)
  y_h_Clerical     ~ dpois(beta_h_Clerical)

  y_l_Management   ~ dpois(beta_l_Management)
  y_h_Management   ~ dpois(beta_h_Management)

  y_l_Other        ~ dpois(beta_l_Other)
  y_h_Other        ~ dpois(beta_h_Other)

  y_l_Professional ~ dpois(beta_l_Professional)
  y_h_Professional ~ dpois(beta_h_Professional)

  y_l_Sales        ~ dpois(beta_l_Sales)
  y_h_Sales        ~ dpois(beta_h_Sales)

  y_l_Service      ~ dpois(beta_l_Service)
  y_h_Service      ~ dpois(beta_h_Service)

  # Priors (vagues, à adapter si tu veux en fonction des counts observés)
  beta_l_Clerical     ~ dgamma(4, 0.1)
  beta_h_Clerical     ~ dgamma(3, 0.1)

  beta_l_Management   ~ dgamma(2, 0.1)
  beta_h_Management   ~ dgamma(3, 0.1)

  beta_l_Other        ~ dgamma(4, 0.1)
  beta_h_Other        ~ dgamma(6, 0.1)

  beta_l_Professional ~ dgamma(3.5, 0.1)
  beta_h_Professional ~ dgamma(3.5, 0.1)

  beta_l_Sales        ~ dgamma(1.5, 0.1)
  beta_h_Sales        ~ dgamma(1, 0.1)

  beta_l_Service      ~ dgamma(3.5, 0.1)
  beta_h_Service      ~ dgamma(1.5, 0.1)

}
"
library(runjags)

jags_samples_occupation <- run.jags(
  model = model_string_occupation,
  data = data_list_occupation,
  monitor = c(
    "beta_l_Clerical", "beta_h_Clerical",
    "beta_l_Management", "beta_h_Management",
    "beta_l_Other", "beta_h_Other",
    "beta_l_Professional", "beta_h_Professional",
    "beta_l_Sales", "beta_h_Sales",
    "beta_l_Service", "beta_h_Service"
  ),
  n.chains = 3,
  adapt = 1000,
  burnin = 1000,
  sample = 5000
)
print(jags_samples_occupation)
plot_credible_intervals(jags_samples_occupation)



In [228]:
analyze_mcmc_difference(jags_samples_occupation, "beta_l_Service", "beta_h_Service")
analyze_mcmc_difference(jags_samples_occupation, "beta_l_Sales", "beta_h_Sales")
analyze_mcmc_difference(jags_samples_occupation, "beta_l_Professional", "beta_h_Professional")
analyze_mcmc_difference(jags_samples_occupation, "beta_l_Other", "beta_h_Other")
analyze_mcmc_difference(jags_samples_occupation, "beta_l_Management", "beta_h_Management")
analyze_mcmc_difference(jags_samples_occupation, "beta_l_Clerical", "beta_h_Clerical")


Les écarts sont plus prononcés dans les professions Management et Other, où les hauts salaires sont significativement plus élevés que les bas salaires. En revanche, Professional et Clerical montrent moins de différence. L'occupation semble donc jouer un rôle différencié selon le secteur en matière d’inégalité salariale.

## MCMC

In [229]:
plot_mcmc(jags_samples_occupation)

## Fake data check

In [231]:
# ------------------------------
# 1. Generate fake Poisson wage counts by occupation and wage level
# ------------------------------
generate_fake_poisson_occupation <- function() {
  # Expected beta values based on the occupation graph
  beta_clerical_l     <- 40
  beta_clerical_h     <- 30
  beta_management_l   <- 20
  beta_management_h   <- 30
  beta_other_l        <- 40
  beta_other_h        <- 60
  beta_professional_l <- 35
  beta_professional_h <- 35
  beta_sales_l        <- 15
  beta_sales_h        <- 10
  beta_service_l      <- 35
  beta_service_h      <- 15

  # Simulate Poisson counts
  list(
    y_l_Clerical     = rpois(1, beta_clerical_l),
    y_h_Clerical     = rpois(1, beta_clerical_h),
    y_l_Management   = rpois(1, beta_management_l),
    y_h_Management   = rpois(1, beta_management_h),
    y_l_Other        = rpois(1, beta_other_l),
    y_h_Other        = rpois(1, beta_other_h),
    y_l_Professional = rpois(1, beta_professional_l),
    y_h_Professional = rpois(1, beta_professional_h),
    y_l_Sales        = rpois(1, beta_sales_l),
    y_h_Sales        = rpois(1, beta_sales_h),
    y_l_Service      = rpois(1, beta_service_l),
    y_h_Service      = rpois(1, beta_service_h)
  )
}

set.seed(123)
fake_occ <- generate_fake_poisson_occupation()

data_list_fake_occupation <- fake_occ
model_string_occupation <- "
model {
  # Likelihoods
  y_l_Clerical     ~ dpois(beta_l_Clerical)
  y_h_Clerical     ~ dpois(beta_h_Clerical)

  y_l_Management   ~ dpois(beta_l_Management)
  y_h_Management   ~ dpois(beta_h_Management)

  y_l_Other        ~ dpois(beta_l_Other)
  y_h_Other        ~ dpois(beta_h_Other)

  y_l_Professional ~ dpois(beta_l_Professional)
  y_h_Professional ~ dpois(beta_h_Professional)

  y_l_Sales        ~ dpois(beta_l_Sales)
  y_h_Sales        ~ dpois(beta_h_Sales)

  y_l_Service      ~ dpois(beta_l_Service)
  y_h_Service      ~ dpois(beta_h_Service)

  # Priors (based on the graph)
  beta_l_Clerical     ~ dgamma(4, 0.1)
  beta_h_Clerical     ~ dgamma(3, 0.1)

  beta_l_Management   ~ dgamma(2, 0.1)
  beta_h_Management   ~ dgamma(3, 0.1)

  beta_l_Other        ~ dgamma(4, 0.1)
  beta_h_Other        ~ dgamma(6, 0.1)

  beta_l_Professional ~ dgamma(3.5, 0.1)
  beta_h_Professional ~ dgamma(3.5, 0.1)

  beta_l_Sales        ~ dgamma(1.5, 0.1)
  beta_h_Sales        ~ dgamma(1, 0.1)

  beta_l_Service      ~ dgamma(3.5, 0.1)
  beta_h_Service      ~ dgamma(1.5, 0.1)
}
"
library(runjags)

jags_samples_occupation_fake <- run.jags(
  model = model_string_occupation,
  data = data_list_fake_occupation,
  monitor = c(
    "beta_l_Clerical", "beta_h_Clerical",
    "beta_l_Management", "beta_h_Management",
    "beta_l_Other", "beta_h_Other",
    "beta_l_Professional", "beta_h_Professional",
    "beta_l_Sales", "beta_h_Sales",
    "beta_l_Service", "beta_h_Service"
  ),
  n.chains = 3,
  adapt = 1000,
  burnin = 1000,
  sample = 5000
)
print(jags_samples_occupation_fake)
plot_credible_intervals(jags_samples_occupation_fake)
summary(jags_samples_occupation_fake)


## Conclusion

Ce tableau compare la valeur moyenne estimée de \( \beta \), c’est-à-dire **le nombre moyen d’individus par catégorie d’occupation** selon le niveau de salaire.

| Occupation     | Salaire       | \( \beta\) réel | \( \beta \) simulé | Écart absolu |
|---------------|--------------|----------------|----------------|--------------|
| Clerical      | < Moyenne    | 40.03          | 36.37          | 3.66         |
| Clerical      | ≥ Moyenne    | 30.17          | 35.39          | 5.22         |
| Management    | < Moyenne    | 19.73          | 12.75          | 6.98         |
| Management    | ≥ Moyenne    | 29.97          | 30.06          |   0.09       |
| Other         | < Moyenne    | 40.04          | 48.99          | 8.95         |
| Other         | ≥ Moyenne    | 60.01          | 62.67          | 2.66         |
| Professional  | < Moyenne    | 34.85          | 27.75          | 7.10         |
| Professional  | ≥ Moyenne    | 35.19          | 24.99          | 10.20        |
| Sales         | < Moyenne    | 14.94          | 18.67          | 3.73         |
| Sales         | ≥ Moyenne    | 9.96           | 10.89          | 0.93         |
| Service       | < Moyenne    | 34.88          | 36.82          | 1.94         |
| Service       | ≥ Moyenne    | 15.12          | 15.00          |   0.12       |

---

### Observations

- **Très bonne cohérence** pour `Service` et `Management` : les données factices reproduisent presque parfaitement les valeurs réelles.
- **Bonne approximation** pour `Sales`, `Clerical`, `Other`, avec de faibles écarts.
- **Sous-estimation marquée** pour `Professional`, particulièrement dans la tranche des hauts salaires.
- **Légère sous-estimation** pour `Management` dans la tranche basse.

---

### Conclusion

Les données simulées **reproduisent bien la structure réelle**, avec des écarts modérés pour certaines catégories.  
Le modèle et les **priors Gamma ajustés à partir des fréquences visuelles** montrent une **bonne performance générale**,  
bien qu’un ajustement des paramètres pour `Professional` pourrait améliorer la précision.


# Références

## Références sur les statistiques bayésiennes  

1. Stats with R. *The Basics of Bayesian Statistics*. Disponible à : [https://statswithr.github.io/book/the-basics-of-bayesian-statistics.html](https://statswithr.github.io/book/the-basics-of-bayesian-statistics.html)  

2. Kruschke, J. *Bayesian Analysis in R*. Disponible à : [https://nyu-cdsc.github.io/learningr/assets/kruschke_bayesian_in_R.pdf](https://nyu-cdsc.github.io/learningr/assets/kruschke_bayesian_in_R.pdf)  

3. Vehtari, A. *Regression and Other Stories — Examples*. Disponible à : [https://avehtari.github.io/ROS-Examples/index.html](https://avehtari.github.io/ROS-Examples/index.html)  

4. Barlaz, M. *Bayesian Portfolio*. Disponible à : [https://marissabarlaz.github.io/portfolio/bayesian/](https://marissabarlaz.github.io/portfolio/bayesian/)  

5. Davis-Ross, K. *Bayesian Reasoning and Methods*. Disponible à : [https://bookdown.org/kevin_davisross/bayesian-reasoning-and-methods/mean.html](https://bookdown.org/kevin_davisross/bayesian-reasoning-and-methods/mean.html)  


6. BayesBall. *Bayesian Multiple Regression and Logistic Models*. Disponible à : [https://bayesball.github.io/BOOK/bayesian-multiple-regression-and-logistic-models.html](https://bayesball.github.io/BOOK/bayesian-multiple-regression-and-logistic-models.html)  


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=650b0311-43bb-4704-b320-0c62dbd2dedc' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>